# Python Control Module

In [136]:
import control as ct
import numpy as np

## State Space Models

The function ss creates a state space model of the system. A continuous linear time-invariant (LTI) system will be used for this example. When you print the system, you will recieve an output that states the type of system, number of inputs/outputs, number of states, and the values of the variables (A, B, C, D) in the system. 

In [137]:
sys = ct.ss(-4, 3, 1, 0)
print(sys)

<LinearIOSystem>: sys[159]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (1): ['x[0]']

A = [[-4.]]

B = [[3.]]

C = [[1.]]

D = [[0.]]



You can also simply type the name of the system to display it in matrix form.

In [138]:
sys

<LinearIOSystem:sys[159]:['u[0]']->['y[0]']>

## Transfer Functions

Creating a transfer function can be accomplished by first defining the constant s using the following command

In [139]:
s = ct.TransferFunction.s
s

TransferFunction(array([1, 0]), array([1]))

This can then be utilized to define a transfer function G(s)

In [140]:
G  = (s + 1)/(s**2 + 2*s + 1)
G

TransferFunction(array([1, 1]), array([1., 2., 1.]))

For single input single output (SISO) systems, the transfer function of a simple system can be defined using the TransferFunction command and specifying the coefficients of the numerator and denominator. 

In [141]:
sys = ct.TransferFunction([2,2], [1, 2, 3])
sys

TransferFunction(array([2, 2]), array([1, 2, 3]))

## Controlability and Observability Matrices

This module has built in functions to easily determine the controllability and observability matrices of a system using the functions ct.ctrb and ct.obs. 

In [142]:
controllability = ct.ctrb(A, B)
controllability

array([[1., 1., 3., 3.],
       [1., 1., 3., 3.]])

In [143]:
observability = ct.obsv(A, C)
observability

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       [2., 1.]])

## Controllability and Observability Gramians

The controllability or observability Gramian can be found using the ct.gram function specifiying the system and type (controllability or observability) of Gramian desired. It is important to note here that using a transfer function model will result in a ValueError as seen below.

In [144]:
ct.gram(G, 'c')

ValueError: System must be StateSpace!

The gramian function will only take a state space respresentation of the model. Furthermore, the system must be stable in order for the Gramian to be computed as seen in the value error below.

In [ ]:
unstableSystem = ct.ss(4, -3, 1, 0)
ct.gram(unstableSystem,'c')

Using the stable LTI system defined earlier, you can successfully compute both the controllability and observability Gramians.

In [ ]:
controllabilityGramian = ct.gram(sys, 'c')
print(f'The controllability Gramian is: {controllabilityGramian}')

In [ ]:
controllabilityGramian = ct.gram(sys, 'o')
print(f'The controllability Gramian is: {controllabilityGramian}')

## Poles,  Zeros, and Gain

The ct.poles and ct.zeros functions are used to determine the poles and zeros of the system. They can be used on both state space representations and transfer functions and return an array of the poles/zeros of the system.

In [ ]:
stateSpacePoles = ct.poles(system)
print(f'The poles of the state space system are {stateSpacePoles}')

transferFunctionPoles = ct.poles(G)
print(f'\nThe poles of the transfer function G are {transferFunctionPoles}')

In [ ]:
stateSpaceZeros = ct.zeros(system)
print(f'The zeros of the state space system are {stateSpaceZeros}')

transferFunctionZeros = ct.zeros(G)
print(f'\nThe zeros of the transfer function G are {transferFunctionZeros}')

The DC gain of the system can also be found using the ct.dcgaini command. Here we will multiply our transfer function G by a gain of 5 to highlight this.

In [ ]:
k = 5 # DC gain 
G = k * (s + 1)/(s**2 + 2*s + 1)
gain = ct.dcgain(G)
print(f'The DC gain of the system is {gain}')

## Block Diagrams

### Series Interconnection 

The series command returns the series connection of two systems sys3=sys1 * sys2.

In [ ]:
H = (s + 2)/(s + 1)
series = ct.series(G, H)
series

### Paralllel Interconnection

The parallel command returns the parallel connection of two systems equivalent to sys3=sys1+sys2.

In [ ]:
parallel = ct.parallel(G, H)
parallel

### Feedback Interconnection

The feedback command gives the feedback interconnection between two I/O systems. The primary plant is sys1. The feedback plant is sys2 (often a feedback controller).

In [ ]:
feedback = ct.feedback(G, H, sign=-1) #the default is sign=-1 which is negative feedback
feedback

## Frequency Domain Plotting

### Bode Plot

The bode plot of the system can be plotted over a (optional) frequency range using the ct.bode command. It returns the magnitude ratio, phase lag, and frequency as well as the bode plot for a given transfer function.

In [ ]:
mag, phase, omega = ct.bode(G)

The bode plot can be customized with several options including setting the x axis to units of Hz or changing the phase from degrees to radians.

In [ ]:
mag, phase, omega = ct.bode(G, w, Hz=True, deg=False)

The freqresp command outputs the same information but does not plot it.

In [ ]:
w=linspace(0.1, 10, 30)
mag, phase, omega = ct.freqresp(G, w)

In [ ]:
mag

### Gain and Phase Margin and Crossover Frequencies

You can calculate the gain and phase margin and the associated crossover frequencies using the margin command.

In [ ]:
gm, pm, Wcg, Wcp = ct.margin(G)
print(f"The gain margin is {gm} degrees with phase crossover frequency {Wcp} rad/s.")
print(f"The phase margin is {pm} with gain crossover frequency {Wcg}.")

### Nyquist Plot

The Nyquist plot for the system can also be plotted over an (optional) frequency range.

In [ ]:
ct.nyquist_plot(G)

## Time Domain Simulation

### Forced Response

The ct.forced_response command simulates the output of a linear system. 

In [ ]:
T=np.linspace(0, 10, 100)
u=2
X0=0 #if left blank, defaults to 0
forced_T, forced_yout = ct.forced_response(G, T, u, X0)
plt.plot(forced_T, forced_yout)

### Impulse Response

The ct.initial_response command gives the response of a linear system given initial conditions. If the system has multiple outputs, one may be selected, or if no selection is made, all outputs are given. Similar to forced response but with zero input.

In [ ]:
impulse_T, impulse_yout = ct.initial_response(G, T, X0) #yout is the response output
plt.plot(impulse_T, impulse_yout)

### Step Response

In [ ]:
step_T, step_yout = ct.step_response(G, T, X0)
plt.plot(step_T, step_yout)